In [23]:
import pandas as pd
import numpy as np
import pandas_profiling as pp
import scipy.stats as stats
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score,mean_squared_error,make_scorer
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import make_classification
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import r2_score,mean_squared_error,make_scorer
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import sys
import os


### Loading data

In [28]:
dflabel=pd.read_csv(os.path.join(os.path.dirname(os.getcwd()), "data/machine_learning_challenge_labeled_data.csv.gz"))
dforders=pd.read_csv(os.path.join(os.path.dirname(os.getcwd()),"data/machine_learning_challenge_order_data.csv.gz"))

In [29]:
dforders.shape

(786600, 13)

In [30]:
dforders.drop_duplicates(keep=False,inplace=True)
dforders.shape

(785571, 13)

### Checking for nulls

In [44]:
dforders.isnull().sum()[dforders.isnull().sum() > 0].reset_index()['index'].values

array(['customer_order_rank'], dtype=object)

In [45]:
dforders['Isnullrank']=np.where(dforders.customer_order_rank.isnull(),1,0)
dforders['Isnonzerovoucher']=np.where(dforders.voucher_amount>0,1,0)

In [33]:
##time frame columns
import calendar

dforders.order_date = pd.to_datetime(dforders.order_date)
dforders['year'] = dforders['order_date'].dt.year
dforders['month'] = dforders.order_date.apply(lambda x: x.strftime('%Y-%m'))
dforders['week_number'] = dforders.order_date.dt.week
dforders['week_day'] = dforders.order_date.dt.day_name()
#dforders['month'] = dforders.month.apply(lambda x: calendar.month_abbr[x])

In [34]:
def day_time(x):
    if x['order_hour'] < 8:
        return 'night'
    elif (x['order_hour'] >=8) & (x['order_hour'] <16):
        return 'day'
    elif x['order_hour'] >= 16:
        return 'evening'
dforders['hour_class'] = dforders.apply(day_time, axis=1)

In [35]:
dfnew=pd.get_dummies(dforders, columns=['hour_class'])

In [36]:
dfnew['Is2017']=np.where(dfnew['year']==2017,1,0)
dfnew['Is2016']=np.where(dfnew['year']==2016,1,0)
dfnew['Is2015']=np.where(dfnew['year']==2015,1,0)

In [52]:
df1=dfnew.groupby('customer_id',as_index=False).agg(['min','max','median'])[['customer_order_rank','voucher_amount','amount_paid','delivery_fee']]

In [53]:
df2=dfnew.groupby(['customer_id']).agg({'order_date': [np.min,np.max]})


In [54]:
df3=dfnew.groupby('customer_id',as_index=False).agg(['sum'])[['customer_order_rank','Isnullrank','Is2017','Is2016','Is2015','is_failed','Isnonzerovoucher','voucher_amount','amount_paid','delivery_fee','hour_class_day','hour_class_evening','hour_class_night']]

In [55]:
df4=dfnew.groupby('customer_id',as_index=False).agg(['nunique'])[['restaurant_id','city_id','payment_id','platform_id','transmission_id','year','week_day','month','order_date']]                           

#### 

In [56]:
df5=dfnew.groupby(['customer_id']).size().reset_index() 
df5.columns=['customer_id','totaltranscations']


,customer_id,totaltranscations
0,000097eabfd9,1
1,0000e2c6d9be,1
2,000133bb597f,1
3,00018269939b,1
4,0001a00468a6,1
...,...,...
245449,fffd696eaedd,1
245450,fffe9d5a8d41,3
245451,ffff347c3cfa,2
245452,ffff4519b52d,1


In [57]:
dforderssummary=df1.merge(df2,on='customer_id').merge(df3,on='customer_id').merge(df4,on='customer_id').merge(df5,on='customer_id').merge(dflabel,on='customer_id')
dforderssummary.head()

c:\users\abinaya.jayakumar\appdata\local\programs\python\python37\lib\site-packages\pandas\core\reshape\merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


,customer_id,"(customer_order_rank, min)","(customer_order_rank, max)","(customer_order_rank, median)","(voucher_amount, min)","(voucher_amount, max)","(voucher_amount, median)","(amount_paid, min)","(amount_paid, max)","(amount_paid, median)",...,"(city_id, nunique)","(payment_id, nunique)","(platform_id, nunique)","(transmission_id, nunique)","(year, nunique)","(week_day, nunique)","(month, nunique)","(order_date, nunique)",totaltranscations,is_returning_customer
0,000097eabfd9,1.0,1.0,1.0,0.0,0.0,0.0,11.46960,11.46960,11.46960,...,1,1,1,1,1,1,1,1,1,0
1,0000e2c6d9be,1.0,1.0,1.0,0.0,0.0,0.0,9.55800,9.55800,9.55800,...,1,1,1,1,1,1,1,1,1,0
2,000133bb597f,1.0,1.0,1.0,0.0,0.0,0.0,5.93658,5.93658,5.93658,...,1,1,1,1,1,1,1,1,1,1
3,00018269939b,1.0,1.0,1.0,0.0,0.0,0.0,9.82350,9.82350,9.82350,...,1,1,1,1,1,1,1,1,1,0
4,0001a00468a6,1.0,1.0,1.0,0.0,0.0,0.0,5.15070,5.15070,5.15070,...,1,1,1,1,1,1,1,1,1,0


In [58]:
cols = dforderssummary.iloc[:,1:-2].columns
newcols = []
for x in cols: 
    newcols.append('_'.join(list(x)))
newcols
newdf = dforderssummary.copy(True)
for i,j in zip(dforderssummary.iloc[:, 1:-2], newcols):
    newdf.rename(columns={i : j}, inplace=True)
newdf.columns

Index(['customer_id', 'customer_order_rank_min', 'customer_order_rank_max',
       'customer_order_rank_median', 'voucher_amount_min',
       'voucher_amount_max', 'voucher_amount_median', 'amount_paid_min',
       'amount_paid_max', 'amount_paid_median', 'delivery_fee_min',
       'delivery_fee_max', 'delivery_fee_median', 'order_date_amin',
       'order_date_amax', 'customer_order_rank_sum', 'Isnullrank_sum',
       'Is2017_sum', 'Is2016_sum', 'Is2015_sum', 'is_failed_sum',
       'Isnonzerovoucher_sum', 'voucher_amount_sum', 'amount_paid_sum',
       'delivery_fee_sum', 'hour_class_day_sum', 'hour_class_evening_sum',
       'hour_class_night_sum', 'restaurant_id_nunique', 'city_id_nunique',
       'payment_id_nunique', 'platform_id_nunique', 'transmission_id_nunique',
       'year_nunique', 'week_day_nunique', 'month_nunique',
       'order_date_nunique', 'totaltranscations', 'is_returning_customer'],
      dtype='object')

In [59]:
newdf['daydiff']=(newdf['order_date_amax']-newdf['order_date_amin'])/np.timedelta64(1, 'D')
newdf['daydiff']

0          0.0
1          0.0
2          0.0
3          0.0
4          0.0
          ... 
245449     0.0
245450    61.0
245451    29.0
245452     0.0
245453     0.0
Name: daydiff, Length: 245454, dtype: float64

In [60]:
newdf['samedaytransaction']=np.where((newdf.order_date_nunique == 1) & (newdf.totaltranscations>1),1,0)
newdf['moretrans']=np.where(newdf.order_date_nunique < newdf.totaltranscations ,1,0)

In [61]:
newdf['customer_order_rank_min'] = newdf['customer_order_rank_min'].replace(np.nan, 0)
newdf['customer_order_rank_max'] = newdf['customer_order_rank_max'].replace(np.nan, 0)
newdf['customer_order_rank_median'] = newdf['customer_order_rank_median'].replace(np.nan, 0)

In [105]:
newdf['order_date_amax'] = pd.to_datetime(newdf.order_date_amax)
maxdate = max(newdf.order_date_amax)
mindate = min(newdf.order_date_amax)
newdf['recenencyscore']= 1- ((maxdate - newdf['order_date_amax']).dt.days / int((maxdate-mindate).total_seconds()/(60*60*24)))

In [106]:
newdf['Is2017binary']=np.where(newdf['Is2017_sum']>0,1,0)
newdf['Is2016binary']=np.where(newdf['Is2016_sum']>0,1,0)
newdf['Is2015binary']=np.where(newdf['Is2015_sum']>0,1,0)


In [107]:
cols=['Is2015binary', 'Is2016binary', 'Is2017binary']
newdf['yearinfo']=newdf[cols].dot(newdf[cols].columns + ';').str.rstrip(';')

In [108]:
newdf['totaltransbinary']=np.where(newdf['totaltranscations']>1,1,0)

In [109]:
newdf.order_date_amax = pd.to_datetime(newdf.order_date_amax)
newdf['yearmax'] = newdf['order_date_amax'].dt.year

In [110]:
newdf.order_date_amin = pd.to_datetime(newdf.order_date_amin)
newdf['yearmin'] = newdf['order_date_amin'].dt.year

In [ ]:
##etl completed

In [111]:
df_2017=newdf[newdf.yearmax==2017]
df_2016=newdf[newdf.yearmax==2016]
df_2015=newdf[newdf.yearmax==2015]
##include other records taht do not follow under this category

In [ ]:
##cluster by year completed

In [ ]:
##build model for 2016

In [112]:
df=df_2016
df_2016.columns

Index(['customer_id', 'customer_order_rank_min', 'customer_order_rank_max',
       'customer_order_rank_median', 'voucher_amount_min',
       'voucher_amount_max', 'voucher_amount_median', 'amount_paid_min',
       'amount_paid_max', 'amount_paid_median', 'delivery_fee_min',
       'delivery_fee_max', 'delivery_fee_median', 'order_date_amin',
       'order_date_amax', 'customer_order_rank_sum', 'Isnullrank_sum',
       'Is2017_sum', 'Is2016_sum', 'Is2015_sum', 'is_failed_sum',
       'Isnonzerovoucher_sum', 'voucher_amount_sum', 'amount_paid_sum',
       'delivery_fee_sum', 'hour_class_day_sum', 'hour_class_evening_sum',
       'hour_class_night_sum', 'restaurant_id_nunique', 'city_id_nunique',
       'payment_id_nunique', 'platform_id_nunique', 'transmission_id_nunique',
       'year_nunique', 'week_day_nunique', 'month_nunique',
       'order_date_nunique', 'totaltranscations', 'is_returning_customer',
       'daydiff', 'samedaytransaction', 'moretrans', 'Is2017binary',
       'Is201

In [113]:
vardf = df[[i for i in df.columns if i not in ['is_returning_customer', 'recenencyscore']]].var().sort_values(ascending=False)
select = vardf.shape[0]//2
final_cols = list(vardf.iloc[:23].index) + ['recenencyscore']
final_cols

['daydiff',
 'customer_order_rank_sum',
 'amount_paid_sum',
 'amount_paid_max',
 'amount_paid_median',
 'amount_paid_min',
 'totaltranscations',
 'customer_order_rank_max',
 'order_date_nunique',
 'hour_class_evening_sum',
 'Is2016_sum',
 'Is2015_sum',
 'customer_order_rank_median',
 'month_nunique',
 'delivery_fee_sum',
 'voucher_amount_sum',
 'restaurant_id_nunique',
 'hour_class_day_sum',
 'week_day_nunique',
 'Isnonzerovoucher_sum',
 'voucher_amount_max',
 'transmission_id_nunique',
 'voucher_amount_median',
 'recenencyscore']

In [114]:
train_df = df[final_cols]
y=df['is_returning_customer']
X=train_df.copy(True)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3)

In [101]:
df.columns

Index(['customer_id', 'customer_order_rank_min', 'customer_order_rank_max',
       'customer_order_rank_median', 'voucher_amount_min',
       'voucher_amount_max', 'voucher_amount_median', 'amount_paid_min',
       'amount_paid_max', 'amount_paid_median', 'delivery_fee_min',
       'delivery_fee_max', 'delivery_fee_median', 'order_date_amin',
       'order_date_amax', 'customer_order_rank_sum', 'Isnullrank_sum',
       'Is2017_sum', 'Is2016_sum', 'Is2015_sum', 'is_failed_sum',
       'Isnonzerovoucher_sum', 'voucher_amount_sum', 'amount_paid_sum',
       'delivery_fee_sum', 'hour_class_day_sum', 'hour_class_evening_sum',
       'hour_class_night_sum', 'restaurant_id_nunique', 'city_id_nunique',
       'payment_id_nunique', 'platform_id_nunique', 'transmission_id_nunique',
       'year_nunique', 'week_day_nunique', 'month_nunique',
       'order_date_nunique', 'totaltranscations', 'is_returning_customer',
       'daydiff', 'samedaytransaction', 'moretrans', 'Is2017binary',
       'Is201

In [97]:
 df[final_cols]

KeyError: "['recenencyscore'] not in index"